## 1. Import libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import datetime

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Any results you write to the current directory are saved as output.

In [3]:
%load_ext tensorboard

In [2]:
# Importing tensorflow and keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Softmax
from tensorflow.keras import optimizers

2023-11-10 13:08:05.086352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Checking the tensorflow libraries  
print(tf.__version__)

2.14.0


# 2. Data exploration

In [10]:
#Loading the dataset

housing_df=pd.read_csv("../../assets/data/california-housing.csv")

In [11]:
housing_df.shape

(20640, 10)

In [12]:
# Displaying the data 

housing_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [13]:
# Checking for the null values 

housing_df.isna().any()

longitude             False
latitude              False
housing_median_age    False
total_rooms           False
total_bedrooms         True
population            False
households            False
median_income         False
median_house_value    False
ocean_proximity       False
dtype: bool

In [10]:
#Count of the null values in each columns

housing_df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [14]:
# We can drop the null values as their count is less than 5 %
housing_df.dropna(inplace=True)

In [15]:
housing_df.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [13]:
housing_df.shape

(20433, 10)

In [16]:
# Dividing the dataset into independant and dependant variables 
features_columns = ['housing_median_age','total_rooms','total_bedrooms','population']
target_columns = ['median_house_value']
X=pd.DataFrame(columns=features_columns,data=housing_df)
y=pd.DataFrame(columns=target_columns,data=housing_df)

In [17]:
housing_df.values[0]

array([-122.23, 37.88, 41.0, 880.0, 129.0, 322.0, 126.0, 8.3252, 452600.0,
       'NEAR BAY'], dtype=object)

In [18]:
X.head()

,housing_median_age,total_rooms,total_bedrooms,population
0,41.0,880.0,129.0,322.0
1,21.0,7099.0,1106.0,2401.0
2,52.0,1467.0,190.0,496.0
3,52.0,1274.0,235.0,558.0
4,52.0,1627.0,280.0,565.0


In [19]:
y.head()

,median_house_value
0,452600.0
1,358500.0
2,352100.0
3,341300.0
4,342200.0


In [20]:
X.head()

,housing_median_age,total_rooms,total_bedrooms,population
0,41.0,880.0,129.0,322.0
1,21.0,7099.0,1106.0,2401.0
2,52.0,1467.0,190.0,496.0
3,52.0,1274.0,235.0,558.0
4,52.0,1627.0,280.0,565.0


# 3. Feature Scaling and test train split

In [21]:
#Dividing the training data into test and train 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


In [22]:
X_train.shape

(14303, 4)

In [23]:
#Feature Standardization

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
scaler.feature_names_in_

array(['housing_median_age', 'total_rooms', 'total_bedrooms',
       'population'], dtype=object)

In [25]:
scaler.scale_

array([1.96078431e-02, 2.54336436e-05, 1.55183116e-04, 2.80292626e-05])

In [26]:
scaler.data_max_, scaler.data_min_, 

(array([   52., 39320.,  6445., 35682.]), array([1., 2., 1., 5.]))

In [27]:
X_train

array([[0.37254902, 0.16099496, 0.15363128, 0.07464193],
       [0.31372549, 0.07192634, 0.07774674, 0.03825994],
       [0.64705882, 0.06704308, 0.08985102, 0.03209351],
       ...,
       [0.43137255, 0.07218068, 0.07603973, 0.04431426],
       [0.15686275, 0.0133018 , 0.02638113, 0.03509264],
       [0.45098039, 0.03214813, 0.04252017, 0.03021555]])

# ANN

In [28]:
model = Sequential()

#Input Layer
model.add(Dense(X.shape[1], activation='relu', input_dim = X.shape[1]))

#Hidden Layer
model.add(Dense(512,kernel_initializer='normal', activation='relu'))
model.add(Dense(512,kernel_initializer='normal', activation='relu'))
model.add(Dense(256,kernel_initializer='normal', activation='relu'))
model.add(Dense(128,kernel_initializer='normal', activation='relu'))
model.add(Dense(64,kernel_initializer='normal', activation='relu'))
model.add(Dense(32,kernel_initializer='normal', activation='relu'))
#Output Layer
model.add(Dense(1,kernel_initializer='normal', activation = 'relu'))


In [29]:
X

,housing_median_age,total_rooms,total_bedrooms,population
0,41.0,880.0,129.0,322.0
1,21.0,7099.0,1106.0,2401.0
2,52.0,1467.0,190.0,496.0
3,52.0,1274.0,235.0,558.0
4,52.0,1627.0,280.0,565.0
...,...,...,...,...
20635,25.0,1665.0,374.0,845.0
20636,18.0,697.0,150.0,356.0
20637,17.0,2254.0,485.0,1007.0
20638,18.0,1860.0,409.0,741.0


In [30]:
X.shape[1]

4

In [31]:
#Compile the network 

model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 512)               2560      
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2

In [32]:
X_train

array([[0.37254902, 0.16099496, 0.15363128, 0.07464193],
       [0.31372549, 0.07192634, 0.07774674, 0.03825994],
       [0.64705882, 0.06704308, 0.08985102, 0.03209351],
       ...,
       [0.43137255, 0.07218068, 0.07603973, 0.04431426],
       [0.15686275, 0.0133018 , 0.02638113, 0.03509264],
       [0.45098039, 0.03214813, 0.04252017, 0.03021555]])

In [33]:
housing_df.values

array([[-122.23, 37.88, 41.0, ..., 8.3252, 452600.0, 'NEAR BAY'],
       [-122.22, 37.86, 21.0, ..., 8.3014, 358500.0, 'NEAR BAY'],
       [-122.24, 37.85, 52.0, ..., 7.2574, 352100.0, 'NEAR BAY'],
       ...,
       [-121.22, 39.43, 17.0, ..., 1.7, 92300.0, 'INLAND'],
       [-121.32, 39.43, 18.0, ..., 1.8672, 84700.0, 'INLAND'],
       [-121.24, 39.37, 16.0, ..., 2.3886, 89400.0, 'INLAND']],
      dtype=object)

In [34]:
log_dir = "../logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [35]:
history = model.fit(X_train, y_train.to_numpy(), batch_size = 10, epochs = 2, verbose = 1, 
                    callbacks=[tensorboard_callback])


Epoch 1/2
1431/1431 [==============================] - 8s 5ms/step - loss: 15391595520.0000 - mse: 15391595520.0000 - mae: 96009.2344
Epoch 2/2
1431/1431 [==============================] - 7s 5ms/step - loss: 12935518208.0000 - mse: 12935518208.0000 - mae: 89255.6719


In [4]:
%tensorboard --logdir ../logs/fit --host localhost --port 8888
# kill $(ps -e | grep 'tensorboard' | awk '{print $1}') -- To kill tensorboard in linux command

In [36]:
X_test

array([[0.70588235, 0.05529274, 0.06471136, 0.03632592],
       [0.37254902, 0.08026858, 0.10599007, 0.06701797],
       [0.19607843, 0.06081184, 0.11250776, 0.0532556 ],
       ...,
       [0.31372549, 0.07586856, 0.09450652, 0.04313704],
       [0.2745098 , 0.03764179, 0.05229671, 0.02954284],
       [1.        , 0.04987538, 0.08752328, 0.04549149]])

In [34]:
y_pred = model.predict(X_test)

192/192 [==============================] - 1s 2ms/step


In [35]:
y_pred

array([[216813.38],
       [186039.55],
       [164051.8 ],
       ...,
       [197880.02],
       [179368.  ],
       [219693.86]], dtype=float32)

In [36]:
y_test

,median_house_value
14185,98900.0
6125,153000.0
14095,91300.0
14359,345200.0
18004,344100.0
...,...
20450,280200.0
14681,264000.0
2639,76800.0
2935,42700.0


In [37]:
model.evaluate(X_test, y_test)

192/192 [==============================] - 1s 2ms/step - loss: 12053440512.0000 - mse: 12053440512.0000 - mae: 86596.1562


[12053440512.0, 12053440512.0, 86596.15625]

In [38]:
# model.save("ann_v1.keras")
# model.save("ann_v1")
model.save("ann/1")

INFO:tensorflow:Assets written to: ann/1/assets


INFO:tensorflow:Assets written to: ann/1/assets


In [39]:
# loaded_model = tf.keras.models.load_model("ann_v1.keras")
loaded_model = tf.keras.models.load_model("ann_v1")

In [40]:
y_pred = loaded_model.predict(X_test)
y_pred

192/192 [==============================] - 1s 2ms/step


array([[211555.88],
       [190811.12],
       [176335.34],
       ...,
       [186958.42],
       [174228.  ],
       [235440.62]], dtype=float32)